In [1]:
%pip install langchain tiktoken langchain-openai langchain-chroma --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
import dotenv
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language


In [2]:
dotenv.load_dotenv()


True

In [22]:
repo_path = "/Users/sunny/projects/onfinance/RAG/data"
loader = GenericLoader.from_filesystem(
    repo_path + "/api-reference/src",
    glob="**/*",
    suffixes=[".ts"],
    parser=LanguageParser(parser_threshold=500),
    # encoding="original_encoding"
)
documents = loader.load()
# utf8_documents = [doc.encode('utf-8') for doc in documents]
len(documents)

95

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
ts_splitter = RecursiveCharacterTextSplitter.from_language(language="ts",chunk_size=250, chunk_overlap=100)
texts = ts_splitter.split_documents(documents)
len(texts)

1232

In [24]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k":8}
)


In [30]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up to get information relevant to the conversation",
        ),
    ]
)

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            " You an AI trained to write code for automation tasks. With respect to the usecase, example and query provided by the user, generate fully functional code to perform the specific task. As there is not max_token limit, generate code entirely, without leaving any place holders, or continuation backticks. Use the reference context examples provided, and try to use trigger.dev in all the cases. :\n\n{context}",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)
document_chain = create_stuff_documents_chain(llm, prompt)

qa = create_retrieval_chain(retriever_chain, document_chain)

In [29]:
question = "Whenever an SQL query returns a blank result, send an email via Gmail based on certain parameters. Whenever the user replies to the email, trigger a webhook to parse and insert gmail response data into the SQL table again. Generate the code, webhooks, automation, etc what ever is needed to perform this task "
result = qa.invoke({"input": question})
print(result["answer"])

Here is the fully functional code to perform the task:

```javascript
const { Client, eventTrigger } = require("@trigger.dev/sdk");
const { google } = require("googleapis");
const z = require("zod");
const axios = require('axios');
const mysql = require('mysql');

const client = new Client();

client.defineJob({
  id: "sql-query-email",
  name: "SQL Query and Send Email",
  version: "1.0.0",
  trigger: eventTrigger({
    name: "sql-query-email",
    schema: z.object({
      sql: z.string(),
    }),
  }),
  integrations: {
    google,
    axios,
    mysql,
  },
  run: async (payload, io, ctx) => {
    const connection = mysql.createConnection({
      host: 'localhost',
      user: 'root',
      password: 'password',
      database: 'database_name'
    });

    connection.query(payload.sql, (err, rows) => {
      if (err) throw err;
      if (!rows.length) {
        const gmail = google.gmail({ version: "v1", auth });
        const message = {
          to: 'recipient@gmail.com',
       